## Problem 1 A. 
Data exploration
Explore the three datasets. Use your judgment as a human being to describe
what makes you able to tell spam apart from ham. Also explore the differences between the two classes of ham (easy and hard ham). What makes them different? Write approximately one paragraph.

1. ##### Difference between Ham and Spam
    Spam contains a lot of clickbaity titles with links that looks more than suspicious. Very sensational language and the senders adress looks very auto-generated for example: sabrina@mx3.1premio.com.
    Ham on the other hand is more legit and the emails looks like they are sent from either real people or real companies with a newsletter and you get the choice to unsubscribe or not.

2. ##### Difference between Hard and Easy Ham
    Easy ham seems to be mostly conversations between real life people, not alot of code, mostly natural speaking language
    Hard Ham contains more code, and contains mostly of commercials which makes it hard to judge if it's actually wanted by the reciever or not.

## Problem 1 B.
Perform an appropriate train-test split on the each of the datasets. We will use
the training sets to train a classifier, and evaluate its performance against the
test sets.

In [8]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

# Creating lists and dicts to parse the text files
easy_ham_texts: list[str] = []
hard_ham_texts: list[str] = []
spam_texts: list[str] = []
easy_ham_labels: list[str] = []
hard_ham_labels: list[str] = []
spam_labels: list[str] = []
easy_ham_dir_path = os.getcwd() + "\\20021010_easy_ham\\easy_ham\\"
hard_ham_dir_path = os.getcwd() + "\\20021010_hard_ham\\hard_ham\\"
spam_dir_path = os.getcwd() + "\\20021010_spam\\spam\\"

# Parsing the text files into lists
for file_name in os.listdir(easy_ham_dir_path):
    f = open(easy_ham_dir_path+file_name, "r", encoding="latin-1")
    text = f.read()
    easy_ham_texts.append(text)
    easy_ham_labels.append("easy_ham")
    f.close()

for file_name in os.listdir(hard_ham_dir_path):
    f = open(easy_ham_dir_path+file_name, "r", encoding="latin-1")
    text = f.read()
    hard_ham_texts.append(text)
    hard_ham_labels.append("hard_ham")
    f.close()

for file_name in os.listdir(spam_dir_path):
    f = open(easy_ham_dir_path+file_name, "r", encoding="latin-1")
    text = f.read()
    spam_texts.append(text)
    spam_labels.append("spam")
    f.close()

# type hinting for the lists
easy_ham_x_test: list[str]
easy_ham_y_test: list[str]
easy_ham_x_train: list[str]
easy_ham_y_train: list[str]
hard_ham_x_test: list[str]
hard_ham_y_test: list[str]
hard_ham_x_train: list[str]
hard_ham_y_train: list[str]
spam_x_test: list[str]
spam_y_test: list[str]
spam_x_train: list[str]
spam_y_train: list[str]

# Make a simple train-test split using 25% of the data for testing as this was recommended in last thursday's lecture,
# no shuffling is needed for naive bayes
easy_ham_x_train, easy_ham_x_test, easy_ham_y_train, easy_ham_y_test = train_test_split(easy_ham_texts, easy_ham_labels, test_size=0.25)

hard_ham_x_train, hard_ham_x_test, hard_ham_y_train, hard_ham_y_test = train_test_split(hard_ham_texts, hard_ham_labels, test_size=0.25)

spam_x_train, spam_x_test, spam_y_train, spam_y_test = train_test_split(spam_texts, spam_labels, test_size=0.25)

print(spam_x_test)
print(spam_y_test)

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\norlandw\\Desktop\\DIT-407-Data-Science-and-AI\\Lab 3\\20021010_easy_ham\\easy_ham\\0001.f0cf04027e74802f09f723cb8916b48e'

## Problem 2.
A bag of words maps text documents into vectors of word counts. That is, each
word is assigned a token number, and we count the number of occurrences of
each unique word in each document. The vector then consists of these counts.
Use the CountVectorizer class in Scikit-Learn to convert the emails to
vectors (or rather the set of emails into a matrix). Acquaint yourself with the
documentation of the class because it is nontrivial to use (specifically understand
what the methods fit, transform, and fit transform do and how to use them
in this setting). In your report, explain succinctly what you did. You don’t have
to separate the headers from the body of the email; processing the entire email
is ok.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix
import numpy as np


vectorizer = CountVectorizer(encoding='latin-1')

# Combine spam and ham into one list for fitting the vectorizer to produce matrixes
vectorizer_fitting_data = []
vectorizer_fitting_data.extend(easy_ham_x_train)
vectorizer_fitting_data.extend(spam_x_train)
vectorizer_fitting_data.extend(hard_ham_x_train)

# Fit the vectorizer to the training data.
vectorizer = vectorizer.fit(vectorizer_fitting_data)

# Transform the training data into a matrix.
easy_ham_x_train_matrix: csr_matrix = vectorizer.transform(easy_ham_x_train)
spam_x_train_matrix: csr_matrix = vectorizer.transform(spam_x_train)
hard_ham_x_train_matrix: csr_matrix = vectorizer.transform(hard_ham_x_train)

# Transform the test data into a matrix.
easy_ham_x_test_matrix: csr_matrix = vectorizer.transform(easy_ham_x_test)
spam_x_test_matrix: csr_matrix = vectorizer.transform(spam_x_test)
hard_ham_x_test_matrix: csr_matrix = vectorizer.transform(hard_ham_x_test)

<class 'scipy.sparse._csr.csr_matrix'>


## Problem 3.
We are going to try out two different Na ̈ıve Bayesian Classifiers: <br>
<a>1https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.</a> <br>
text.CountVectorizer.html

* The Multinomial Naive Bayesian Classifier, and
* The Bernoulli Naive Bayesian Classifier.

For this problem, use the set easy ham as your ham. Use the training sets
of ham and spam to train an instance of both classifiers. Then, evaluate the
classifier against the test set.
In your report, report the accuracy, precision, and recall of your classifiers,
that is, six numbers. Also provide the confusion matrix for both classifiers (2×2
matrix where positive/negative predictions are the columns, and actual posi-
tive/negative values as rows). The count of true/false positives and true/false
negatives should be easily readable in the matrix, together with the marginal
sums.

In [ ]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import precision_score, recall_score
import numpy as np

# Create the training data.
training_data = np.concatenate((easy_ham_x_train_matrix.toarray(), spam_x_train_matrix.toarray()))
training_labels = np.concatenate((easy_ham_y_train, spam_y_train))

# Create the classifiers and fit it to the training data.
multinomial_nb_clf = MultinomialNB()
multinomial_nb_clf.fit(training_data, training_labels)

bernoulli_nb_clf = BernoulliNB()
bernoulli_nb_clf.fit(training_data, training_labels)

# Create the test data.
test_data = np.concatenate((easy_ham_x_test_matrix.toarray(), spam_x_test_matrix.toarray()))
test_labels = np.concatenate((easy_ham_y_test, spam_y_test))

# Evaluate accuracy on the test data.
# accuracy answers the question: how often the model is right?
bernoulli_accuracy_score = bernoulli_nb_clf.score(test_data, test_labels)
multinomial_accuracy_score = multinomial_nb_clf.score(test_data, test_labels)

print(f"MultinomialNB accuracy: {multinomial_accuracy_score}")
print(f"BernoulliNB accuracy: {bernoulli_accuracy_score}")

# Evaluate the precision, figured average = "binary" is fine since this is a two class classification problem.
# precision answers the question: how often the positive predictions are correct?
easy_ham_precision_multinomial = precision_score(easy_ham_y_test, multinomial_nb_clf.predict(easy_ham_x_test_matrix), pos_label="easy_ham")
spam_precision_multinomial = precision_score(spam_y_test, multinomial_nb_clf.predict(spam_x_test_matrix), pos_label="spam")
easy_ham_precision_bernoulli = precision_score(easy_ham_y_test, bernoulli_nb_clf.predict(easy_ham_x_test_matrix), pos_label="easy_ham")
spam_precision_bernoulli = precision_score(spam_y_test, bernoulli_nb_clf.predict(spam_x_test_matrix), pos_label="spam")

print(f"MultinomialNB easy_ham precision: {easy_ham_precision_multinomial}")
print(f"MultinomialNB spam precision: {spam_precision_multinomial}")
print(f"BernoulliNB easy_ham precision: {easy_ham_precision_bernoulli}")
print(f"BernoulliNB spam precision: {spam_precision_bernoulli}")

# Evaluate the recall 
# recall answers the question: can an ML model find all instances of the positive class?
easy_ham_recall_multinomial = recall_score(easy_ham_y_test, multinomial_nb_clf.predict(easy_ham_x_test_matrix), pos_label="easy_ham", zero_division=0)
spam_recall_multinomial = recall_score(spam_y_test, multinomial_nb_clf.predict(spam_x_test_matrix), pos_label="spam", zero_division=0)
easy_ham_recall_bernoulli = recall_score(easy_ham_y_test, bernoulli_nb_clf.predict(easy_ham_x_test_matrix), pos_label="easy_ham", zero_division=0)
spam_recall_bernoulli = recall_score(spam_y_test, bernoulli_nb_clf.predict(spam_x_test_matrix), pos_label="spam", zero_division=0)

print(f"MultinomialNB easy_ham recall: {easy_ham_recall_multinomial}")
print(f"MultinomialNB spam recall: {spam_recall_multinomial}")
print(f"BernoulliNB easy_ham recall: {easy_ham_recall_bernoulli}")
print(f"BernoulliNB spam recall: {spam_recall_bernoulli}")


